### Machine Learning: naive Bayes classifier for continuous data from scratch in Python
$P(C_k|\textbf{x})\propto P(C_k) P(\textbf{x}|C_k)$
###### by Hamed Shah-Hosseini
Explanation at: https://www.pinterest.com/HamedShahHosseini/
<br>https://github.com/ostad-ai/Machine-Learning

In [18]:
import pandas as pd
from math import pi,sqrt,exp

In [19]:
# Reading dataset
iris=pd.read_csv('./datasets/iris-uci.csv')
print(f'Number of samples in dataset:{len(iris)}')
print(f'Number of classes: {iris["Class"].unique().shape[0]}')
print(f'Features:{iris.columns[:-1].to_list()}')

Number of samples in dataset:150
Number of classes: 3
Features:['Sepal-length', ' Sepal-width', 'Petal-length', 'Petal-width']


In [20]:
# The first five rows of dataset
iris.head()

,Sepal-length,Sepal-width,Petal-length,Petal-width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [27]:
# Naive Bayes for continuous data
class NBContinuous:
    def __init__(self):
        pass
    def fit(self,data_frame,class_column):
        df=data_frame
        self.groups_m_v=df.groupby(class_column).agg(['mean','var'])
        self.features=df.columns.to_list()
        self.features.remove(class_column)
        self.priors=(pd.value_counts(df[class_column])/len(df)).to_dict() #prior  
        self.cNames=df[class_column].unique() #class categories
    def gaussian(self,x,mean,var):
        return (1./sqrt(2*pi*var))*exp(-(x-mean)**2/(2*var))
    def predict(self,query,with_priors=True):
        probs_classes=dict()        
        for cName in self.cNames:
            if with_priors:
                prob=self.priors[cName]
            else:
                prob=1.
            for q,feature in zip(query,self.features):
                    mean=self.groups_m_v.loc[cName][feature]['mean']
                    var=self.groups_m_v.loc[cName][feature]['var']
                    prob*=self.gaussian(q,mean,var) 
            probs_classes[cName]=prob
        predict=max(probs_classes,key=probs_classes.get)
        return predict,probs_classes

In [28]:
# Experiment with the Naive Baye's Classifier
classCol='Class'
class_names=list(iris[classCol].unique())
# Splitting dataset into training and test sets
iris_train=iris.sample(frac=.8)
iris_test=iris.drop(iris_train.index)
Nts=iris_test.shape[0] #number of test samples
columns=iris.columns.to_list()
classCol_inx=columns.index(classCol)
nbayes=NBContinuous()
# Training with training set
nbayes.fit(iris_train,classCol)
# Testing with test set
accuracy=0.; actuals=[]; predicteds=[]
for n in range(Nts):
    query=iris_test.iloc[n].to_list()
    #remove the value of class column
    class_q=query.pop(classCol_inx)
    actuals.append(class_q)
    predicted=nbayes.predict(query)[0]
    predicteds.append(predicted)
    accuracy+=predicted==class_q
accuracy/=Nts   
print('Accuracy:%',round(100*accuracy,2))
#-Computing confusion matrix
Nclasses=len(class_names)
confusion_mat=pd.DataFrame(0,index=class_names,
                        columns=class_names)
for n in range(Nts):
    confusion_mat.loc[actuals[n],predicteds[n]]+=1
confusion_mat
#--Could use pd.crosstab for confusion matrix

Accuracy:% 96.67


,Iris-setosa,Iris-versicolor,Iris-virginica
Iris-setosa,13,0,0
Iris-versicolor,0,8,1
Iris-virginica,0,0,8
